# 5주차 스터디

Made by 35기 코딩부장 류제현

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

## Pandas 응용과 정규표현식(Regular Expression) 소개

    Spam 메일 데이터 셋을 사용하겠습니다

In [2]:
# 데이터를 불러옵니다
spam = pd.read_csv("spam.csv")

# 정답지(target)을 1과 0으로 변경 (머신러닝 모델을 짜고 싶다면 이렇게 변경해야 한다)
spam['target'] = np.where(spam['target']=='spam',1,0)

spam

,text,target
0,"Go until jurong point, crazy.. Available only ...",0
1,Ok lar... Joking wif u oni...,0
2,Free entry in 2 a wkly comp to win FA Cup fina...,1
3,U dun say so early hor... U c already then say...,0
4,"Nah I don't think he goes to usf, he lives aro...",0
...,...,...
5567,This is the 2nd time we have tried 2 contact u...,1
5568,Will Ì_ b going to esplanade fr home?,0
5569,"Pity, * was in mood for that. So...any other s...",0
5570,The guy did some bitching but I acted like i'd...,0


데이터 전체에서 spam의 비율이 몇 퍼센트일까요

In [3]:
sum(spam.target == 1) / len(spam.target)

0.13406317300789664

spam의 문자열 수와, non_spam의 문자열 개수의 평균을 계산해봅시다

In [4]:
avg_notspam = spam.query('target == 0')['text'].apply(len).values.mean()
avg_spam = spam.query('target == 1')['text'].apply(len).values.mean()    

avg_notspam, avg_spam

(71.02362694300518, 138.8661311914324)

숫자가 평균적으로 몇 번 들어가 있을까요

* 정규표현식 소개 (Regular Expression)

In [5]:
string = "한 주당 1,000원, 시가총액은 300,000,000원입니다"

In [6]:
import re

re.findall("[0-9,]*원", string)

['1,000원', '300,000,000원']

In [7]:
# Spam 매일과 Non-spam 매일에 들어있는 숫자의 개수 차가 통계적으로 유의미한 지 확인해 봅시다
def cal_digit(x):
    pure_digit = re.findall('[0-9]',x)
    len_digit = len(pure_digit)
    return len_digit

digit_1 = spam.query("target==1")["text"].apply(cal_digit)
digit_0 = spam.query("target==0")["text"].apply(cal_digit)

# 숫자의 평균 계산
mean_1 = digit_1.mean()
mean_0 = digit_0.mean()

# 숫자의 표준편차 계산
std_1 = digit_1.std()
std_0 = digit_0.std()

print("평균: ", mean_1, mean_0)
print("표준편차: ", std_1, std_0)
print("표본의 수: ", len(digit_0), len(digit_1))

평균:  15.759036144578314 0.2992746113989637
표준편차:  8.755896252067858 1.063085509449761
표본의 수:  4825 747


$CI = \bar{X} \pm z \times {\sigma \over \sqrt n}$

In [8]:
# T-test를 해보자
import statsmodels.api as sm 

sm.stats.ttest_ind(digit_1, digit_0)

(117.24448159629607, 0.0, 5570.0)

## 싸이킷런

In [9]:
from sklearn.datasets import load_wine
df = load_wine(as_frame=True) # 3가지 종류의 와인

X = df['data']
y = df['target']

# 멀티 클래스 회귀를 단일 클래스 회귀로 변경하겠습니다
y_binary = (y == 1).astype('int')

In [10]:
X.head()

,alcohol,malic_acid,ash,alcalinity_of_ash,magnesium,total_phenols,flavanoids,nonflavanoid_phenols,proanthocyanins,color_intensity,hue,od280/od315_of_diluted_wines,proline
0,14.23,1.71,2.43,15.6,127.0,2.80,3.06,0.28,2.29,5.64,1.04,3.92,1065.0
1,13.20,1.78,2.14,11.2,100.0,2.65,2.76,0.26,1.28,4.38,1.05,3.40,1050.0
2,13.16,2.36,2.67,18.6,101.0,2.80,3.24,0.30,2.81,5.68,1.03,3.17,1185.0
3,14.37,1.95,2.50,16.8,113.0,3.85,3.49,0.24,2.18,7.80,0.86,3.45,1480.0
4,13.24,2.59,2.87,21.0,118.0,2.80,2.69,0.39,1.82,4.32,1.04,2.93,735.0


In [11]:
# 결측치 있는 지 확인
X.isnull().sum().sum() , y.isna().sum().sum()   # na와 null은 동일하다, 둘 중 하나가 더 익숙한 사람을 위해 만들어 둔 것

(0, 0)

In [12]:
# 훈련셋과 테스트 셋 분리해주기
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X,y_binary, train_size=0.8) # default 0.75
X_train.shape, X_test.shape

((142, 13), (36, 13))

### 분류기 평가 Metric 


* 정확도 / 정밀도 / 재현율

In [13]:
# 간단하게 모형 만들어보기
from sklearn.tree import DecisionTreeClassifier

tree_model = DecisionTreeClassifier()
tree_model.fit(X_train, y_train) 

DecisionTreeClassifier()

In [14]:
# Tree기반 모델은 변수 중요도(feature_importance)를 확인할 수 있습니다
tree_model.feature_importances_

array([0.12854147, 0.        , 0.02865278, 0.        , 0.01884874,
       0.        , 0.        , 0.        , 0.        , 0.70258949,
       0.04045584, 0.08091168, 0.        ])

In [15]:
tree_model.feature_names_in_ # petal은 꽃잎

array(['alcohol', 'malic_acid', 'ash', 'alcalinity_of_ash', 'magnesium',
       'total_phenols', 'flavanoids', 'nonflavanoid_phenols',
       'proanthocyanins', 'color_intensity', 'hue',
       'od280/od315_of_diluted_wines', 'proline'], dtype=object)

In [16]:
# 분류기의 경우에 score는 단순한 정확도를 리턴 (accuracy score)
tree_model.score(X_train, y_train)

# 단일 Tree는 무조건 오버피팅 하기 때문에, 정확도가 1.0이 나오는 것을 확인할 수 있다.

1.0

In [17]:
# test-set에 대한 점수를 확인해 봅시다
tree_model.score(X_test,y_test)

1.0

목표로 하는 task에 따라서 중요하게 생각해야하는 Metric이 달라진다 (통계학의 1종오류, 2종오류 개념과 유사)

* 정확도(Accuracy)  : 전체 데이터 수 중 예측 결과와 실제 값이 동일한 건수(TN + TP)가 차지하는 비율

* 정밀도(Precision) : 예측을 Positive로 한 대상(FP + TP) 중 예측과 실제 값이 Positive로 일치한 데이터(TP)의 비율

* 재현율(Recall) : 실제가 Positive인 대상(FN + TP) 중 예측과 실제 값이 Positive로 일치한 데이터(TP)의 비율



In [18]:
from sklearn.metrics import confusion_matrix

confusion_matrix(y_test,tree_model.predict(X_test)) # 실제는 positive인데, 예측을 negative로 한 값이 하나 존재한다

array([[17,  0],
       [ 0, 19]], dtype=int64)

In [19]:
# 분류의 경우 정확도 말고도 사용할 수 있는 다양한 성능평가 지표들이 있습니다. 
## 그 중에서 정밀도와 재현율을 배워보겠습니다

from sklearn.metrics import precision_score, recall_score

print("모델의 정확도: ", tree_model.score(X_test,y_test))
print("모델의 정밀도: ", precision_score(y_test, tree_model.predict(X_test)))
print("모델의 재현율: ", recall_score(y_test, tree_model.predict(X_test)))

모델의 정확도:  1.0
모델의 정밀도:  1.0
모델의 재현율:  1.0


### Cross-Validation 소개

* Test-set을 미리 들여다 보는 우를 범하지 않기 위해 사용함

In [20]:
# 훈련하고 성과 측정하기 
from sklearn.model_selection import cross_val_score

tree_model = DecisionTreeClassifier()

# cv=5이면 데이터를 5번 쪼갠다. 데이터는 반드시 Random해야 한다!
cv_score = cross_val_score(tree_model, X_train, y_train, cv=5, n_jobs=-1, verbose=True)

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 16 concurrent workers.
[Parallel(n_jobs=-1)]: Done   5 out of   5 | elapsed:    3.5s finished


In [21]:
print(cv_score)

# cv_score의 평균
cv_score.mean()

[0.93103448 0.82758621 0.89285714 0.96428571 0.96428571]


0.9160098522167488

### Grid-Search 소개

* Grid-Search
* Random-search

In [22]:
from sklearn.model_selection import GridSearchCV

tree_model = DecisionTreeClassifier()

param_grid = {"criterion" : ["gini","entropy"],  
              'max_depth': [10,20,30,40,50]} 

model_reg = GridSearchCV(tree_model, param_grid, cv=10, verbose=True, n_jobs=-1)
model_reg.fit(X_train,y_train)

model_reg.best_params_

Fitting 10 folds for each of 10 candidates, totalling 100 fits


{'criterion': 'gini', 'max_depth': 40}

In [23]:
model_reg.score(X_train, y_train)

1.0

In [24]:
# Random Search
from sklearn.model_selection import RandomizedSearchCV

random_forest = DecisionTreeClassifier()
param_grid = {'max_depth':range(10,40)}

random_reg = RandomizedSearchCV(random_forest, param_grid, n_iter=10, cv=5, n_jobs=-1, verbose=True)
random_reg.fit(X_train, y_train)

Fitting 5 folds for each of 10 candidates, totalling 50 fits


RandomizedSearchCV(cv=5, estimator=DecisionTreeClassifier(), n_jobs=-1,
                   param_distributions={'max_depth': range(10, 40)},
                   verbose=True)

In [25]:
random_reg.best_estimator_

DecisionTreeClassifier(max_depth=35)